# Form Recognizer で領収書を分析します

![領収書を持っているロボット](./images/receipt_analysis.jpg)

Computer Vision の人工知能 (AI) の分野では、印刷文書や手書き文書を読み取るために光学式文字認識 (OCR) が一般的に使用されます。多くの場合、それらの文書からテキストを単に抽出したあと、さらなる処理や分析は抽出先のフォーマットを使用して行われます。

より高度な OCR のシナリオには、注文書や請求書などのフォームから情報を抽出し、それと同時にフォームの各フィールドが表す情報を意味論的に理解することです。**Form Recognizer** サービスは、AI のこの種の課題に対応できるように特別に設計されています。

## 領収書を表示する

この例では、Form Recognizer の組み込みモデルを使用して領収書を分析します。

セルの左側にある 「**セルの実行**」(&#9655;) ボタンをクリックして実行し、Form Recognizer を使用して分析する領収書の例を確認します。

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline


# Load and display a receipt image
fig = plt.figure(figsize=(6, 6))
image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## Form Recognizer リソースを作成する

Azure サブスクリプションに Form Recognizer リソースを作成することから始めます。

1. ブラウザーの新しいタブで Azure portal (https://portal.azure.com) を開き、Microsoft アカウントでサインインします。
2. 「**+ リソースの作成**」 を選択し、*Form Recognizer* を検索します。
3. サービスの一覧から 「**Form Recognizer**」 を選択します。
4. 「**Form Recognizer**」 ブレードで 「**作成**」 を選択します。
5. 「**作成**」 ブレードで次の詳細を入力し、「**作成**」 を選択します。
   - **名前**: サービスの一意の名前
   - **サブスクリプション**: 使用する Azure サブスクリプション
   - **リージョン**: 利用可能な任意のリージョン
   - **価格レベル**: F0
   - **リソース グループ**: 前に使用した既存のリソース グループ
   - **注意事項を読み理解しました**: 選択されています。
6. サービスが作成されるまで待ちます。
7. 新しく作成した Form Recognizer サービスを Azure portal で表示し、「**キーとエンドポイント**」 ページから 「**キー1**」 と 「**エンドポイント**」 の値をコピーして以下のコード セルに貼り付けます (**YOUR_FORM_KEY**、**YOUR_FORM_ENDPOINT** とそれぞれ置き換える)。

In [ ]:
form_key = 'YOUR_FORM_KEY'
form_endpoint = 'YOUR_FORM_ENDPOINT'

print('Ready to use form recognizer at {} using key {}'.format(form_endpoint, form_key))

## 領収書を分析する

これで、Form Recognizer を使用して領収書を分析する準備が整いました。

In [ ]:
import os
from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential

# Create a client for the form recognizer service
form_recognizer_client = FormRecognizerClient(endpoint=form_endpoint, credential=AzureKeyCredential(form_key))

try:
    print("Analyzing receipt...")
    # Get the receipt image file
    image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')

    # Submit the file data to form recognizer
    with open(image_path, "rb") as f:
        analyze_receipt = form_recognizer_client.begin_recognize_receipts(receipt=f)
    
    # Get the results
    receipt_data = analyze_receipt.result()

    # Print the extracted data for the first (and only) receipt
    receipt = receipt_data[0]
    receipt_type = receipt.fields.get("ReceiptType")
    if receipt_type:
        print("Receipt Type: {}".format(receipt_type.value))
    merchant_address = receipt.fields.get("MerchantAddress")
    if merchant_address:
        print("Merchant Address: {}".format(merchant_address.value))
    merchant_phone = receipt.fields.get("MerchantPhoneNumber")
    if merchant_phone:
        print("Merchant Phone: {}".format(merchant_phone.value))
    transaction_date = receipt.fields.get("TransactionDate")
    if transaction_date:
        print("Transaction Date: {}".format(transaction_date.value))
    print("Receipt items:")
    items = receipt.fields.get("Items")
    if items:
        for idx, item in enumerate(receipt.fields.get("Items").value):
            print("\tItem #{}".format(idx+1))
            item_name = item.value.get("Name")
            if item_name:
                print("\t - Name: {}".format(item_name.value))
            item_total_price = item.value.get("TotalPrice")
            if item_total_price:
                print("\t - Price: {}".format(item_total_price.value))
    subtotal = receipt.fields.get("Subtotal")
    if subtotal:
        print("Subtotal: {} ".format(subtotal.value))
    tax = receipt.fields.get("Tax")
    if tax:
        print("Tax: {}".format(tax.value))
    total = receipt.fields.get("Total")
    if total:
        print("Total: {}".format(total.value))

except Exception as ex:
    print('Error:', ex)


Form Recognizer は、フォーム内のデータを解釈して、店舗の住所と電話番号、販売日時、各行の項目、小計、税金、合計金額などを正しく識別することができます。

## 詳細情報

Form Recognizer サービスの詳細については、「[Form Recognizer ドキュメント](https://docs.microsoft.com/ja-jp/azure/cognitive-services/form-recognizer/index)」を参照してください。